In [26]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [34]:
column_names = [
    "name", "start_date", "start_time", "duration", "reverseDelta", "IoTMac", "hostMac", 
    "IoTIP", "IoTIP_int", "hostIP", "hostIP_int", "ipProto", "IoTPort", "hostPort", 
    "PacketCount", "BytesCount", "reversePacketCount", "reverseBytesCount", "SmallPktCount", 
    "LargePktCount", "NonEmptyPktCount", "DataByteCount", "AvgIAT", "FirstNonEmptyPktSize", 
    "MaxPktSize", "StdevPayloadSize", "StdevIAT", "AvgPacketSize", "reverseSmallPktCount", 
    "reverseLargePktCount", "reverseNonEmptyPktCount", "reverseDataByteCount", "reverseAvgIAT", 
    "reverseFirstNonEmptyPktSize", "reverseMaxPktSize", "reverseStdevPayloadSize", "reverseStdevIAT", 
    "reverseAvgPacketSize", "reverseFlowExists", "remote", "broadcast", "HTTP", "HTTPS", "DNS", 
    "NTP", "TCP_others", "UDP_others", "year_month"
]

# Load data and clean 'name' column
df = pd.read_csv('sample.csv', names=column_names)
df['name'] = df['name'].str.replace(r'_\d{4}_\d{2}\.csv$', '', regex=True)

# Encode 'name' as dummy variables
df = pd.get_dummies(df, columns=['name'], drop_first=True)

# Drop columns that won't be used as features
df = df.drop(['IoTMac', 'hostMac', 'start_date', 'start_time'], axis=1)

df.head()

,duration,reverseDelta,IoTIP,IoTIP_int,hostIP,hostIP_int,ipProto,IoTPort,hostPort,PacketCount,...,name_philips_hue_bridge,name_planex_camera_one_shot!,name_planex_smacam_outdoor,name_planex_smacam_pantilt,name_powerelectric_wi-fi_plug,name_qrio_hub,name_sony_bravia,name_sony_network_camera,name_sony_smart_speaker,name_xiaomi_mijia_led
0,0.000,0.000,192.168.1.236,3232236012,239.255.255.250,4026531834,17,33033,1900,1,...,False,True,False,False,False,False,False,False,False,False
1,0.000,0.000,192.168.1.184,3232235960,192.168.1.1,3232235777,17,33789,53,1,...,False,False,False,False,False,False,False,False,False,False
2,0.000,0.000,192.168.1.236,3232236012,239.255.255.250,4026531834,17,46713,1900,1,...,False,True,False,False,False,False,False,False,False,False
3,0.434,0.012,192.168.1.124,3232235900,74.125.102.41,1249732137,6,53460,443,18,...,False,False,False,False,False,False,True,False,False,False
4,0.000,0.000,192.168.1.236,3232236012,239.255.255.250,4026531834,17,48008,1900,1,...,False,True,False,False,False,False,False,False,False,False


In [52]:
# just to get i have every category

target_columns = [col for col in df.columns if col not in column_names]

len(target_columns)
target_columns

['name_au_network_camera',
 'name_au_wireless_adapter',
 'name_bitfinder_awair_breathe_easy',
 'name_candy_house_sesami_wi-fi_access_point',
 'name_google_home_gen1',
 'name_i-o_data_qwatch',
 'name_irobot_roomba',
 'name_jvc_kenwood_cu-hb1',
 'name_jvc_kenwood_hdtv_ip_camera',
 'name_line_clova_wave',
 'name_link_japan_eremote',
 'name_mouse_computer_room_hub',
 'name_panasonic_doorphone',
 'name_philips_hue_bridge',
 'name_planex_camera_one_shot!',
 'name_planex_smacam_outdoor',
 'name_planex_smacam_pantilt',
 'name_powerelectric_wi-fi_plug',
 'name_qrio_hub',
 'name_sony_bravia',
 'name_sony_network_camera',
 'name_sony_smart_speaker',
 'name_xiaomi_mijia_led',
 'year',
 'month']

In [48]:
# string_columns = df.select_dtypes(include='object').columns
# print("Columns with string data:", string_columns)
# it prints Columns with string data: Index(['IoTIP', 'hostIP', 'year_month'], dtype='object'.
# so here is to convert them

import ipaddress

# Convert IP addresses to integer format for numerical representation
df['IoTIP'] = df['IoTIP'].apply(lambda x: int(ipaddress.IPv4Address(x)))
df['hostIP'] = df['hostIP'].apply(lambda x: int(ipaddress.IPv4Address(x)))


# Parse the 'year_month' column into a datetime format
df['year_month'] = pd.to_datetime(df['year_month'], format='%Y_%m')

# extract year and month into separate columns
df['year'] = df['year_month'].dt.year
df['month'] = df['year_month'].dt.month

# Drop the original 'year_month' column if it's no longer needed
df = df.drop(columns=['year_month'])

In [50]:

X = df.drop(target_columns, axis=1)
y = df[target_columns]

# Split the dataset into training and test sets 20-80
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.96


In [54]:
X.columns


Index(['duration', 'reverseDelta', 'IoTIP', 'IoTIP_int', 'hostIP',
       'hostIP_int', 'ipProto', 'IoTPort', 'hostPort', 'PacketCount',
       'BytesCount', 'reversePacketCount', 'reverseBytesCount',
       'SmallPktCount', 'LargePktCount', 'NonEmptyPktCount', 'DataByteCount',
       'AvgIAT', 'FirstNonEmptyPktSize', 'MaxPktSize', 'StdevPayloadSize',
       'StdevIAT', 'AvgPacketSize', 'reverseSmallPktCount',
       'reverseLargePktCount', 'reverseNonEmptyPktCount',
       'reverseDataByteCount', 'reverseAvgIAT', 'reverseFirstNonEmptyPktSize',
       'reverseMaxPktSize', 'reverseStdevPayloadSize', 'reverseStdevIAT',
       'reverseAvgPacketSize', 'reverseFlowExists', 'remote', 'broadcast',
       'HTTP', 'HTTPS', 'DNS', 'NTP', 'TCP_others', 'UDP_others', 'year',
       'month'],
      dtype='object')

In [56]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation

# Step 3: Print the results
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())

Cross-validation scores: [0.958  0.963  0.9565 0.959  0.9665]
Average cross-validation score: 0.9606
Standard deviation of cross-validation scores: 0.0036523964735499436
